# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [80]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [81]:
spark = SparkSession.builder.appName('Consultingproj').getOrCreate()

In [82]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema = True, header = True)

In [83]:
df.describe()

DataFrame[summary: string, Ship_name: string, Cruise_line: string, Age: string, Tonnage: string, passengers: string, length: string, cabins: string, passenger_density: string, crew: string]

In [84]:
for temp in df.head(2)[1]:
    print(temp)

Quest
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [85]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [86]:
#converting the data types 
from pyspark.sql.types import IntegerType

In [87]:
df = df.withColumn("Tonnage",df["Tonnage"].cast(IntegerType()))
df = df.withColumn("passengers",df["passengers"].cast(IntegerType()))
df = df.withColumn("length",df["length"].cast(IntegerType()))
df = df.withColumn("cabins",df["cabins"].cast(IntegerType()))
df = df.withColumn("passenger_density",df["passenger_density"].cast(IntegerType()))
df = df.withColumn("crew",df["crew"].cast(IntegerType()))


In [88]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: integer (nullable = true)
 |-- passengers: integer (nullable = true)
 |-- length: integer (nullable = true)
 |-- cabins: integer (nullable = true)
 |-- passenger_density: integer (nullable = true)
 |-- crew: integer (nullable = true)



In [89]:
df.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30, passengers=6, length=5, cabins=3, passenger_density=42, crew=3)

In [90]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [91]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [92]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'],outputCol = 'features')

In [93]:
output = assembler.transform(df)

In [94]:
output.head(3)[1]

Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30, passengers=6, length=5, cabins=3, passenger_density=42, crew=3, features=DenseVector([6.0, 30.0, 6.0, 5.0, 3.0, 42.0]))

In [95]:
Final_data =output.select('features','crew')

In [96]:
Final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.0,6.0,5.0...|   3|
|[6.0,30.0,6.0,5.0...|   3|
|[26.0,47.0,14.0,7...|   6|
|[11.0,110.0,29.0,...|  19|
|[17.0,101.0,26.0,...|  10|
|[22.0,70.0,20.0,8...|   9|
|[15.0,70.0,20.0,8...|   9|
|[23.0,70.0,20.0,8...|   9|
|[19.0,70.0,20.0,8...|   9|
|[6.0,110.0,37.0,9...|  11|
|[10.0,110.0,29.0,...|  11|
|[28.0,46.0,14.0,7...|   6|
|[18.0,70.0,20.0,8...|   9|
|[17.0,70.0,20.0,8...|   9|
|[11.0,86.0,21.0,9...|   9|
|[8.0,110.0,29.0,9...|  11|
|[9.0,88.0,21.0,9....|  10|
|[15.0,70.0,20.0,8...|   9|
|[12.0,88.0,21.0,9...|   9|
|[20.0,70.0,20.0,8...|   9|
+--------------------+----+
only showing top 20 rows



In [97]:
train_data,test_data = train_data,test_data = Final_data.randomSplit([0.7,0.3])

In [98]:
train_data.head(5)

[Row(features=DenseVector([4.0, 220.0, 54.0, 11.0, 27.0, 40.0]), crew=21),
 Row(features=DenseVector([5.0, 86.0, 21.0, 9.0, 10.0, 40.0]), crew=8),
 Row(features=DenseVector([5.0, 115.0, 35.0, 9.0, 15.0, 32.0]), crew=12),
 Row(features=DenseVector([5.0, 133.0, 39.0, 10.0, 16.0, 33.0]), crew=13),
 Row(features=DenseVector([5.0, 160.0, 36.0, 11.0, 18.0, 44.0]), crew=13)]

In [99]:
from pyspark.ml.regression import LinearRegression

In [100]:
lr = LinearRegression(labelCol = 'crew')

In [101]:
lrfit = lr.fit(train_data)

In [102]:
lrtest = lrfit.evaluate(test_data)


In [103]:
TestResults = lrtest.predictions
lrtest.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.12067516602536088|
| -0.7532860832596135|
| 0.17563191965704483|
| -1.5158739789250095|
| -1.6095502609198657|
| -0.3003583142371671|
| -0.9070815257635694|
| -0.6426388671344281|
| -0.5529401443199067|
| -0.6021698224019172|
|-0.13638381741929972|
| -0.8627219209637236|
|  0.8841371180096207|
| -0.7348164854146821|
|-0.09591477268678883|
| 0.08839721539911594|
| -1.0767014320063257|
| -0.7856743915660349|
| -1.5451401369196827|
|  0.5233827543874874|
+--------------------+
only showing top 20 rows



In [104]:
lrtest.r2

0.854792875193789